In [5]:
import numpy as np
import pandas as pd
from faker import Faker
from scipy.stats import skewnorm
from datetime import datetime  # datetime 모듈 추가

# Faker 객체 생성
fake = Faker('ko_KR')

# 원본 데이터의 통계 정보 (예시 값, 실제 데이터로 대체 필요)
num_samples = 994731  # 생성할 데이터 샘플 수

# 거래금액 통계 정보 (예시 값)
tran_amt_mean = 802473.2905198361
tran_amt_std = 5172857.605729902
tran_amt_skewness = 33.89506349779731

wd_fc_sn_min = 101
wd_fc_sn_max = 167

dps_fc_sn_min = 101
dps_fc_sn_max = 167

wd_ac_sn_min = 200000000000001
wd_ac_sn_max = 200000002121354

dps_ac_sn_min = 200000000000002
dps_ac_sn_max = 200000002121353

#=== 금융사기의심 통계 ===
ff_sp_ai_ratio_normal = 0.9989191044370099  # 정상 거래 비율 (NaN 포함)
ff_sp_ai_ratio_01 = 0.0004297645583875113  # 사기 의심 01 비율
ff_sp_ai_ratio_02 = 0.0004000077608944813  # 사기 의심 02 비율
ff_sp_ai_ratio_SP = 0.00025112324370807094  # 금융사기 의심거래 비율

# 1. 통계 정보를 기반으로 데이터 생성
def generate_skewed_data(mean, std, skewness, n_samples):
    # 왜도(skewness)를 고려한 데이터 생성
    a = skewness
    data = skewnorm.rvs(a, loc=mean, scale=std, size=n_samples)
    return np.clip(data, a_min=0, a_max=None)  # 음수 방지를 위해 클리핑

# 거래금액 생성
tran_amt = generate_skewed_data(tran_amt_mean, tran_amt_std, tran_amt_skewness, num_samples)

# 2. 기타 데이터 생성
# 거래날짜 (2023년 1월 1일부터 2023년 12월 31일까지)
tran_dt = [
    fake.date_between(
        start_date=datetime(2023, 1, 1),  # datetime 객체로 변환
        end_date=datetime(2023, 12, 31)   # datetime 객체로 변환
    ).strftime('%Y%m%d') 
    for _ in range(num_samples)
]

# 거래시간범위 (0, 3, 6, 9, 12, 15, 18, 21)
tran_tmrg = np.random.choice([0, 3, 6, 9, 12, 15, 18, 21], size=num_samples)

# 출금금융기관코드 (60개의 범주)
wd_fc_sn = np.random.randint(wd_fc_sn_min, wd_fc_sn_max, size=num_samples, dtype=np.int64)

# 출금계좌번호 (996481개의 계좌)
wd_ac_sn = np.random.randint(wd_ac_sn_min, wd_ac_sn_max, size=num_samples, dtype=np.int64)

# 입금금융기관코드 (60개의 범주)
dps_fc_sn = np.random.randint(dps_fc_sn_min, dps_fc_sn_max, size=num_samples, dtype=np.int64)

# 입금계좌번호 (1161975개의 계좌)
dps_ac_sn = np.random.randint(dps_ac_sn_min, dps_ac_sn_max, size=num_samples, dtype=np.int64)

# 이체방법 (1, 2, 3, 4, 5, 6, 7)
md_type = np.random.randint(1, 8, size=num_samples, dtype=np.int64)

# 자금 유형 (0: 일반, 1:급여, 2:배당금, 3:기타, 4: 타행자동이체, 롤 추첨)
fnd_type = np.random.choice([0, 1, 2, 3, 4, 11, 12, 13, 14, 15, 16, 18, 23], size=num_samples)

# 금융사기의심 (결측치: NaN, 01: 사기 의심, 02: 사기 의심, SP: 금융사기 의심거래)
ff_sp_ai = np.random.choice(
    [np.nan, '01', '02', 'SP'],  # null 값은 np.nan으로 처리
    size=num_samples,
    p=[ff_sp_ai_ratio_normal, ff_sp_ai_ratio_01, ff_sp_ai_ratio_02, ff_sp_ai_ratio_SP]
)

# 3. 데이터프레임 생성
df = pd.DataFrame({
    'tran_dt': tran_dt,
    'tran_tmrg': tran_tmrg,
    'wd_fc_sn': wd_fc_sn,
    'wd_ac_sn': wd_ac_sn,
    'dps_fc_sn': dps_fc_sn,
    'dps_ac_sn': dps_ac_sn,
    'tran_amt': tran_amt,
    'md_type': md_type,
    'fnd_type': fnd_type,
    'ff_sp_ai': ff_sp_ai
})

# 4. 데이터 확인
print(df.head())

# 5. CSV 파일로 저장
df.to_csv("synthetic_HF_TRNS_TRAN.csv", index=False)
print("합성 데이터가 성공적으로 저장되었습니다.")

    tran_dt  tran_tmrg  wd_fc_sn         wd_ac_sn  dps_fc_sn        dps_ac_sn  \
0  20230526         12       145  200000001863982        164  200000001159196   
1  20230914         15       156  200000000566910        127  200000000109302   
2  20230731         18       121  200000001469307        146  200000000171226   
3  20230718          6       123  200000000542230        152  200000001583155   
4  20231203          3       151  200000001085979        159  200000000051157   

       tran_amt  md_type  fnd_type ff_sp_ai  
0  4.248310e+06        4        11      nan  
1  2.725604e+06        1        16      nan  
2  5.287491e+06        3         3      nan  
3  2.580183e+06        2         3      nan  
4  1.783105e+06        7         0      nan  
합성 데이터가 성공적으로 저장되었습니다.


In [6]:
import os
import pandas as pd

pd.set_option('display.max_columns', None)
path = './synthetic_HF_TRNS_TRAN.csv'
df = pd.read_csv(path)

df.shape

(994731, 10)

In [7]:
df.head()

,tran_dt,tran_tmrg,wd_fc_sn,wd_ac_sn,dps_fc_sn,dps_ac_sn,tran_amt,md_type,fnd_type,ff_sp_ai
0,20230526,12,145,200000001863982,164,200000001159196,4.248310e+06,4,11,NaN
1,20230914,15,156,200000000566910,127,200000000109302,2.725604e+06,1,16,NaN
2,20230731,18,121,200000001469307,146,200000000171226,5.287491e+06,3,3,NaN
3,20230718,6,123,200000000542230,152,200000001583155,2.580183e+06,2,3,NaN
4,20231203,3,151,200000001085979,159,200000000051157,1.783105e+06,7,0,NaN
